In [12]:
import pdfplumber

In [13]:
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [14]:
from PIL import Image
from io import BytesIO
from transformers import pipeline

In [15]:
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pdfplumber
import pytesseract
from PIL import Image
from io import BytesIO
from transformers import pipeline

# Function to extract text from the PDF, including text from images and tables
def extract_text_from_pdf(pdf_path):
    pdf_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            # Extract regular text
            pdf_text += page.extract_text() or ""
            
            # Extract tables and add to the text
            tables = page.extract_tables()
            for table in tables:
                pdf_text += "\n" + " | ".join([" | ".join(row) for row in table if row]) + "\n"
            
            # Extract images and run OCR
            for img in page.images:
                try:
                    # Get the image as a byte array
                    x0, top, x1, bottom = img['x0'], img['top'], img['x1'], img['bottom']
                    image = page.within_bbox((x0, top, x1, bottom)).to_image()

                    # Extract the original image as a PIL image
                    pil_image = image.original
                    
                    # Convert PIL Image to bytes and apply OCR
                    img_byte_arr = BytesIO()
                    pil_image.save(img_byte_arr, format='PNG')  # Save to bytes
                    img_byte_arr.seek(0)  # Move to the beginning of the BytesIO buffer
                    ocr_text = pytesseract.image_to_string(Image.open(img_byte_arr))
                    pdf_text += "\n[Image Text]: " + ocr_text + "\n"
                except Exception as e:
                    print(f"Error processing image: {e}")
    return pdf_text

# Function to summarize long text in chunks
def summarize_long_text(text, chunk_size=1000, max_length=150, min_length=50):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    # Split text into chunks of chunk_size
    text_chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    
    # Summarize each chunk
    summaries = []
    for chunk in text_chunks:
        # Adjust max_length dynamically
        input_length = len(chunk)
        adjusted_max_length = min(max_length, input_length // 2)
        summary = summarizer(chunk, max_length=adjusted_max_length, min_length=min_length, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    
    # Combine summaries
    full_summary = " ".join(summaries)
    return full_summary

# Main function to extract and summarize the PDF content
def summarize_pdf(pdf_path, chunk_size=1000, max_length=150, min_length=50):
    # Step 1: Extract text, tables, and images (with OCR) from the PDF
    pdf_text = extract_text_from_pdf(pdf_path)
    
    # Step 2: Summarize the extracted content
    summary = summarize_long_text(pdf_text, chunk_size=chunk_size, max_length=max_length, min_length=min_length)
    
    return summary

# Provide the path to your PDF
pdf_path = "C:/Users/nisht/OneDrive/Desktop/default Project/Problem Statement.pdf" # Adjust this if the file is not in the same directory
summary = summarize_pdf(pdf_path, chunk_size=1000, max_length=150, min_length=50)
print("Summary of the PDF:")
print(summary)

Error processing image: Bounding box (476.4, 836.2099999999999, 566.4, 875.2099999999999) is not fully within parent page bounding box (0, 0.0, 595.32, 841.92)


c:\python 311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your max_length is set to 150, but your input_length is only 100. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Summary of the PDF:
Prajna AI hackathon challenge focuses on building acomprehensive PDF ingestion and querying system that allows users to upload PDF documents. The solution should feature an intuitive, interactive frontend and be deployable in a cloud environment. Detailed Requirements and Criteria:. PDF Document Ingestion10 MARKS. User Querying Interface: Allow users to submit natural language queries against the PDF. Citation and Validation: Provide citations for the information returned in response to userqueries. Interactive Frontend: Design a user-friendly web interface for document interaction. 2. Embedding Generation and Data Persistence: Use a state-of-the-art model to generate embeddings that capture the semantics of the ARTICLEPDF content. 3. Question Suggestion Engine: Develop an engine that generates 3-5 insightful questions related to the uploaded ARTICLEPDF’s content. 4. User Querying Interface: Create a responsive interface that allows users to submit natural language 